In [ ]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [1]:
import pandas as pd

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
print(df["koi_disposition"].unique())

['CONFIRMED' 'FALSE POSITIVE' 'CANDIDATE']


# Select your features (columns)

In [4]:
# Set X equal to the entire data set, except for the first column
X = df.iloc[:,1:]

# Set y equal to the first column
y = df.iloc[:,0]

In [5]:
# Search for top 10 features according to feature importances
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X,y)
model.feature_importances_

array([0.1315498 , 0.16826816, 0.12515083, 0.05270868, 0.02016454,
       0.0149871 , 0.01405626, 0.01209883, 0.02119528, 0.01980315,
       0.01214584, 0.01151967, 0.00958516, 0.01718146, 0.02531344,
       0.03493906, 0.02130118, 0.01345864, 0.01340111, 0.01307731,
       0.0117729 , 0.01203832, 0.01639716, 0.00913053, 0.00859724,
       0.00808117, 0.03131429, 0.00925959, 0.00999121, 0.02023611,
       0.01620453, 0.01063699, 0.01023477, 0.01385829, 0.00896986,
       0.01020115, 0.00928137, 0.01123069, 0.01030645, 0.0103519 ])

In [6]:
# Store the top 10 features as a series, using the column headers as the index
feat_imp = pd.Series(model.feature_importances_, index=X.columns).nlargest(10)
feat_imp

koi_fpflag_ss        0.168268
koi_fpflag_nt        0.131550
koi_fpflag_co        0.125151
koi_fpflag_ec        0.052709
koi_duration_err2    0.034939
koi_model_snr        0.031314
koi_duration_err1    0.025313
koi_depth            0.021301
koi_time0bk_err1     0.021195
koi_steff_err1       0.020236
dtype: float64

In [8]:
# Set features based on feature importances
X = df[feat_imp.index]

# Use koi_disposition for y values
y = df['koi_disposition']

# Create a Train Test Split

Use `koi_disposition` for the y values

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [10]:
X_train.head()

,koi_fpflag_ss,koi_fpflag_nt,koi_fpflag_co,koi_fpflag_ec,koi_duration_err2,koi_model_snr,koi_duration_err1,koi_depth,koi_time0bk_err1,koi_steff_err1
6467,0,1,0,0,-1.40000,12.0,1.40000,499.7,0.04610,165
6454,1,0,0,0,-0.00653,4563.5,0.00653,72895.0,0.00009,84
6699,0,0,0,0,-0.58800,9.2,0.58800,50.9,0.02220,142
4123,1,0,0,1,-0.24700,12.0,0.24700,33.1,0.01160,166
5558,0,0,0,1,-0.25400,19.2,0.25400,18.2,0.00936,175


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [11]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler object
X_scaler = MinMaxScaler().fit(X_train)

# Scale training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [12]:
from sklearn.linear_model import LogisticRegression

# Initialize model
model = LogisticRegression()

# Train the model
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [14]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8498950982262063
Testing Data Score: 0.8489702517162472


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [15]:
predictions = model.predict(X_test_scaled)
df_pred = pd.DataFrame({"Actual":y_test, "Predicted":predictions}) 
df_pred.head()

,Actual,Predicted
4341,CANDIDATE,CANDIDATE
4401,FALSE POSITIVE,FALSE POSITIVE
4878,CANDIDATE,CANDIDATE
1423,CANDIDATE,CANDIDATE
575,FALSE POSITIVE,FALSE POSITIVE


In [17]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
import numpy as np

# Create the GridSearchCV model
param_grid = {'C':np.logspace(-4, 4, 20),
             'penalty':['l1','l2']}

grid = GridSearchCV(model, param_grid, verbose=3)

In [18]:
# Train the model with GridSearch
# Train the model with GridSearch
best_model = grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5] END ................C=0.0001, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END ................C=0.0001, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END ................C=0.0001, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END ................C=0.0001, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END ................C=0.0001, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END ..............C=0.0001, penalty=l2;, score=0.501 total time=   0.0s
[CV 2/5] END ..............C=0.0001, penalty=l2;, score=0.501 total time=   0.0s
[CV 3/5] END ..............C=0.0001, penalty=l2;, score=0.501 total time=   0.0s
[CV 4/5] END ..............C=0.0001, penalty=l2;, score=0.501 total time=   0.0s
[CV 5/5] END ..............C=0.0001, penalty=l2;, score=0.501 total time=   0.0s
[CV 1/5] END C=0.00026366508987303583, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END C=0.00026366508987303583, penalty=

[CV 2/5] END ...C=1.623776739188721, penalty=l2;, score=0.863 total time=   0.1s
[CV 3/5] END ...C=1.623776739188721, penalty=l2;, score=0.850 total time=   0.0s
[CV 4/5] END ...C=1.623776739188721, penalty=l2;, score=0.865 total time=   0.0s
[CV 5/5] END ...C=1.623776739188721, penalty=l2;, score=0.843 total time=   0.0s
[CV 1/5] END .....C=4.281332398719396, penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END .....C=4.281332398719396, penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END .....C=4.281332398719396, penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END .....C=4.281332398719396, penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END .....C=4.281332398719396, penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END ...C=4.281332398719396, penalty=l2;, score=0.844 total time=   0.2s
[CV 2/5] END ...C=4.281332398719396, penalty=l2;, score=0.869 total time=   0.2s
[CV 3/5] END ...C=4.281332398719396, penalty=l2;, score=0.856 total time=   0.0s
[CV 4/5] END ...C=4.28133239

In [19]:
print(best_model.best_params_)
print('Best Score: ', best_model.best_score_)

{'C': 1438.44988828766, 'penalty': 'l2'}
Best Score:  0.8624817165020848


In [20]:
# Make predictions
grid_predictions = best_model.predict(X_test_scaled)
df_grid = pd.DataFrame({"Actual":y_test, "Predicted":grid_predictions}) 
df_grid.head()

,Actual,Predicted
4341,CANDIDATE,CANDIDATE
4401,FALSE POSITIVE,FALSE POSITIVE
4878,CANDIDATE,CANDIDATE
1423,CANDIDATE,CANDIDATE
575,FALSE POSITIVE,FALSE POSITIVE


In [21]:
# Score the model
best_model.score(X_test_scaled, y_test)

0.8524027459954233

# Save the Model

In [22]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'LR00.sav'
joblib.dump(best_model, filename)

['LR00.sav']